<a href="https://colab.research.google.com/github/Anurag38/NLP-Practicals/blob/main/Practical%205%20-%20LSA/LSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.decomposition import TruncatedSVD
# If nltk stop word is not downloaded
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# List of documents
a1 = "He is a good dog."
a2 = "The dog is too lazy."
a3 = "That is a brown cat."
a4 = "The cat is very active."
a5 = "I have brown cat and dog."

df = pd.DataFrame()
df["documents"] = [a1,a2,a3,a4,a5]
df.head()


,documents
0,He is a good dog.
1,The dog is too lazy.
2,That is a brown cat.
3,The cat is very active.
4,I have brown cat and dog.


In [3]:
# Preprocessing
df['clean_documents'] = df['documents'].str.replace("[^a-zA-Z#]", " ")
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: x.lower())

df.head()

,documents,clean_documents
0,He is a good dog.,good dog
1,The dog is too lazy.,the dog too lazy
2,That is a brown cat.,that brown cat
3,The cat is very active.,the cat very active
4,I have brown cat and dog.,have brown cat and dog


In [4]:
import nltk 
nltk.download('stopwords') 
from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# tokenization
tokenized_doc = df['clean_documents'].fillna('').apply(lambda x: x.split())

# remove stop-words

#tokenized_doc = tokenized_doc.apply(lambda x: [itemstop_words = stopwords.words('english') for item in x if item not in stop_words])
# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) 
# de-tokenization
detokenized_doc = []
for i in range(len(df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

df['clean_documents'] = detokenized_doc


In [6]:
df.head()

,documents,clean_documents
0,He is a good dog.,good dog
1,The dog is too lazy.,dog lazy
2,That is a brown cat.,brown cat
3,The cat is very active.,cat active
4,I have brown cat and dog.,brown cat dog


In [7]:
# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(df['clean_documents'])
X.toarray()


array([[0.        , 0.        , 0.        , 0.55645052, 0.83088075,
        0.        ],
       [0.        , 0.        , 0.        , 0.55645052, 0.        ,
        0.83088075],
       [0.        , 0.76944707, 0.63871058, 0.        , 0.        ,
        0.        ],
       [0.83088075, 0.        , 0.55645052, 0.        , 0.        ,
        0.        ],
       [0.        , 0.64846263, 0.53828256, 0.53828256, 0.        ,
        0.        ]])

In [8]:
X.shape 
#A56   U(5,5). S()

(5, 6)

In [9]:
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(X)


In [10]:
#Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded_df["documents"] = df['clean_documents']
display(topic_encoded_df[["documents", "topic_1", "topic_2"]])


,documents,topic_1,topic_2
0,good dog,0.3413834191239963,0.7199781067501040
1,dog lazy,0.3413834191239966,0.7199781067501028
2,brown cat,0.8609490919302167,-0.3659836550739515
3,cat active,0.5166658991993207,-0.3850046207843261
4,brown cat dog,0.9494117370834869,0.0236302940661148


In [11]:
# Features or words used as features 
dictionary = vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
dictionary

['active', 'brown', 'cat', 'dog', 'good', 'lazy']

In [13]:
# Term-Topic matrix
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2"], columns = (dictionary)).T


In [14]:
encoding_matrix

,topic_1,topic_2
active,0.2003541259081108,-0.2424408501618362
brown,0.5965117122287049,-0.2018098984872580
cat,0.6293380994160952,-0.3298859088715317
dog,0.4158307960649448,0.6169033286639758
good,0.1323826028466491,0.4533766476433699
lazy,0.1323826028466496,0.4533766476433685
